In [1]:
!git clone https://github.com/Rope-player/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 548 (delta 5), reused 162 (delta 0), pack-reused 374
Receiving objects: 100% (548/548), 50.13 MiB | 29.28 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd "pytorch_advanced"

/content/pytorch_advanced


In [3]:
%cd "3_semantic_segmentation"

/content/pytorch_advanced/3_semantic_segmentation


In [4]:
import os
import urllib.request
import zipfile
import tarfile

In [5]:
data_dir = "./data/"
if not os.path.exists(data_dir):
  os.mkdir(data_dir)

In [6]:
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
  os.mkdir(weights_dir)

In [7]:
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
  urllib.request.urlretrieve(url, target_path)
    
  tar = tarfile.TarFile(target_path)  
  tar.extractall(data_dir)  
  tar.close()

In [8]:
import os.path as osp
from PIL import Image

import torch.utils.data as data

# 시맨틱 분할

## 화상 데이터 및 어노테이션 파일의 경로 리스트 작성

In [9]:
def make_datapath_list(rootpath):
  # 학습 및 검증을 위한 화상 데이터와 어노테이션 데이터 파일 경로 리스트 작성
  # Parameters
  #   rootpath : str - 데이터 폴더 경로
  # Returns
  #   ret : train_img_list, train_anno_list, val_img_list, val_anno_list - 데이터 경로를 지정한 리스트

  # 화상 파일과 어노테이션 파일의 경로 템플릿 작성
  imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
  annopath_template = osp.join(rootpath, 'SegmentationClass', '%s.png')

  # 훈련 및 검증 파일 각각으 ID 취득
  train_id_names = osp.join(rootpath + 'ImageSets/Segmentation/train.txt')
  val_id_names = osp.join(rootpath + 'ImageSets/Segmentation/val.txt')

  # 훈련 데이터의 화상과 어노테이션의 경로 리스트 작성
  train_img_list = list()
  train_anno_list = list()

  for line in open(train_id_names):
    file_id = line.strip()                     # 공백과 줄바꿈 제거
    img_path = (imgpath_template % file_id)    # 화상 경로
    anno_path = (annopath_template % file_id)  # 어노테이션 경로
    train_img_list.append(img_path)
    train_anno_list.append(anno_path)

  # 검증 데이터의 화상 파일과 어노테이션의 경로 리스트 작성
  val_img_list = list()
  val_anno_list = list()

  for line in open(val_id_names):
    file_id = line.strip()  
    img_path = (imgpath_template % file_id)  
    anno_path = (annopath_template % file_id)
    val_img_list.append(img_path)
    val_anno_list.append(anno_path)

  return train_img_list, train_anno_list, val_img_list, val_anno_list

In [10]:
# 동작 확인
rootpath = "./data/VOCdevkit/VOC2012/"

train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

print(train_img_list[0])
print(train_anno_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg
./data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png


## 데이터셋 작성

화상과 어노태이션을 전처리하는 `DataTransform` 클래스 작성후 `Dataset` 클래스 작성. 먼저 화상 데이터와 어노테이션을 데이터 세트로 변환.

이어서 훈련데이터의 데이터 확장. 화상 크기를 `Scale` 클래스로 변형. 크기가 커졌다면 잘라내고 작아졌다면 여백을 검은색으로 채움.

그리고 `RandomRoataion` 클래스로 회전을 시키고 `RandomMirror` 클래스로 좌우를 반전 시킴.

`Resize` 클래스로 지정된 화상크기로 변환하고, `Normalize_Tensor` 클래스로 텐서형식으로 변환 시킴.



검증 데이터는 데이터 확장을 실시하지 않고 `Resize` 클래스부터 시작함

In [11]:
from utils.data_augumentation import Compose, Scale, RandomRotation, RandomMirror, Resize, Normalize_Tensor

class DataTransform():
  # 화상과 어노테이션의 전처리 클래스, 훈려시와 검증시 다르게 동작.
  # 화상 크기를 input_sizexinput_size로 함. 훈련 시 데이터 학장 수행
  #
  # Attributes
  #   input_size : int       - 리사이즈 대상 크기
  #   color_mean : (R, G, B) - 각 색상 채널의 평균 값
  #   color_std  : (R, G, B) - 각 색상 채널의 표준 편차

  def __init__(self, input_size, color_mean, color_std):
    self.data_transform = {
      'train': Compose([
        Scale(scale=[0.5, 1.5]),                 # 확대, 축소
        RandomRotation(angle=[-10, 10]),         # 회전
        RandomMirror(),                          # 랜덤 반전
        Resize(input_size),                      # 리사이즈(input_size)
        Normalize_Tensor(color_mean, color_std)  # 색상 정보의 표준화와 텐서화
      ]),
      'val': Compose([
        Resize(input_size),                      # 리사이즈(input_size)
        Normalize_Tensor(color_mean, color_std)  # 색상 정보의 표준화와 텐서화
      ])
    }
  
  def __call__(self, phase, img, anno_class_img):
    # Parameters
    #   phase : 'train' or 'val' - 전처리 모드 지정.

    return self.data_transform[phase](img, anno_class_img)

Dataset 클래스인 VOCDataset 클래스 작성

In [12]:
class VOCDataset(data.Dataset):
    # VOC2012のDatasetを作成するクラス。PyTorchのDatasetクラスを継承。
    #
    # Attributes
    #   img_list  : 리스트 - 어노테이션 경로 리스트
    #   anno_list : 리스트 - 어노테이션 경로 리스트
    #   phase     : 'train' or 'test' - 학습, 훈련 설정
    #   transform : object - 전처리 클래스 인스턴스

    def __init__(self, img_list, anno_list, phase, transform):
      self.img_list = img_list
      self.anno_list = anno_list
      self.phase = phase
      self.transform = transform

    def __len__(self):
      # 화상 매수 반환
      return len(self.img_list)

    def __getitem__(self, index):
      # 전처리한 화상의 텐서 형식 데이터와 어노테이션 취득
      img, anno_class_img = self.pull_item(index)
      return img, anno_class_img

    def pull_item(self, index):
      # 화상의 텐서 형식 데이터와 어노테이션 취득

      # 1. 화상 읽기
      image_file_path = self.img_list[index]
      img = Image.open(image_file_path)   # [높이][폭][RGB]

      # 2. 어노테이션 화상 읽기
      anno_file_path = self.anno_list[index]
      anno_class_img = Image.open(anno_file_path)   # [높이][폭]

      # 3. 전처리 실시
      img, anno_class_img = self.transform(self.phase, img, anno_class_img)

      return img, anno_class_img

In [13]:
# 동작 확인

# (RGB)의 평균치와 표준편차
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

# 데이터셋 작성
train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

# 데이터 추출 예
print(val_dataset.__getitem__(0)[0].shape)
print(val_dataset.__getitem__(0)[1].shape)
print(val_dataset.__getitem__(0))

torch.Size([3, 475, 475])
torch.Size([475, 475])
(tensor([[[ 1.6667,  1.5125,  1.5639,  ...,  1.7523,  1.6667,  1.7009],
         [ 1.5810,  1.4269,  1.4783,  ...,  1.7009,  1.6153,  1.6495],
         [ 1.5639,  1.4098,  1.4440,  ...,  1.6838,  1.5982,  1.6324],
         ...,
         [-0.4739, -0.4911, -0.5424,  ...,  1.2557,  1.1872,  1.2214],
         [-0.5596, -0.4911, -0.4911,  ...,  1.2385,  1.1872,  1.2214],
         [-0.6281, -0.3883, -0.3369,  ...,  1.2385,  1.1872,  1.2214]],

        [[ 1.8333,  1.6758,  1.7283,  ...,  1.9209,  1.8333,  1.8683],
         [ 1.7458,  1.5882,  1.6408,  ...,  1.8683,  1.7808,  1.8158],
         [ 1.7283,  1.5707,  1.6057,  ...,  1.8508,  1.7633,  1.7983],
         ...,
         [-0.5826, -0.6001, -0.6527,  ...,  1.4132,  1.3431,  1.3431],
         [-0.6702, -0.6001, -0.6001,  ...,  1.3957,  1.3431,  1.3431],
         [-0.7402, -0.4951, -0.4426,  ...,  1.3957,  1.3431,  1.3431]],

        [[ 2.0474,  1.8905,  1.9428,  ...,  2.1346,  2.0474,  2.08

## 데이터 로더 작성

In [14]:
batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 사전 오브젝트로 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 동작 확인
batch_iterator = iter(dataloaders_dict["val"])  # 반복자로 변환
imges, anno_class_imges = next(batch_iterator)  # 첫 번째 요소 꺼내기
print(imges.size())                             # torch.Size([8, 3, 475, 475])
print(anno_class_imges.size())                  # torch.Size([8, 3, 475, 475])

torch.Size([8, 3, 475, 475])
torch.Size([8, 475, 475])
